In [28]:
from pathlib import Path
import duckdb
import pandas as pd
# Define the path to the Parquet file
crs_file = Path("~/Downloads") / "CRS.parquet"
dac1_file = Path("~/Downloads") / "Table1_Data.csv"

data_save_path = Path("../") 

DAC_COUNTRIES = [
  "Australia", "Austria", "Belgium", "Canada", "Czechia", "Denmark", "Finland", "France", "Germany", "Greece", "Hungary",
  "Iceland", "Ireland", "Italy", "Japan", "Korea", "Luxembourg", "Netherlands", "New Zealand", "Norway", "Poland",
  "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States", "Estonia", "Lithuania"
]

../DT_update


## ONE Campaign

In [ ]:
# from oda_data import ODAData, set_data_path

# # set the path to the folder where the data should be stored
# set_data_path("data")

# # create object, specifying key details of the desired output
# data = ODAData(years=range(2018,2023), currency="EUR", prices="constant", base_year=2021)

# # load the desired indicators
# data.load_indicator(indicators = ["imputed_multi_flow_disbursement_gross", "total_oda_ge"])

# # get the data
# df = data.get_data()

# print(df.head(6))

INFO 2024-12-13 22:04:40,563 [read.py:read_multisystem:138] Multisystem data not found. Downloading...
INFO: Multisystem data not found. Downloading...
INFO 2024-12-13 22:04:40,563 [multisystem.py:download_multisystem:34] Downloading Multisystem data... This may take a while.
INFO: Downloading Multisystem data... This may take a while.
INFO: Downloading parquet file, which will be returned as a DataFrame. This may take a while...
INFO: Reading 1 files.
INFO: File downloaded correctly.
INFO 2024-12-13 22:05:53,527 [multisystem.py:save_clean_multisystem:22] multisystem_raw data downloaded and saved.
INFO: multisystem_raw data downloaded and saved.
INFO 2024-12-13 22:05:54,994 [read.py:read_crs:60] CRS data not found. Downloading...
INFO: CRS data not found. Downloading...
INFO 2024-12-13 22:05:54,995 [crs.py:download_crs:31] The full, detailed CRS is only available as a large file (>1GB). The package will now download the data, but it may take a while.
INFO: The full, detailed CRS is onl

   year                              indicator  donor_code  recipient_code  \
0  2020  imputed_multi_flow_disbursement_gross           1              55   
1  2020  imputed_multi_flow_disbursement_gross           1              55   
2  2020  imputed_multi_flow_disbursement_gross           1              55   
3  2020  imputed_multi_flow_disbursement_gross           1              55   
4  2020  imputed_multi_flow_disbursement_gross           1              55   
5  2020  imputed_multi_flow_disbursement_gross           1              55   

   purpose_code  channel_code currency    prices     value  
0         11110         42003      EUR  constant  0.695669  
1         11120         41122      EUR  constant  0.000026  
2         11120         42003      EUR  constant   0.08108  
3         11182         41122      EUR  constant  0.000004  
4         11220         42003      EUR  constant  0.083913  
5         11230         41119      EUR  constant  0.000063  


## CRS DATA

In [2]:
with duckdb.connect() as conn:

    cols_query = f"""
    SELECT * FROM '{crs_file.as_posix()}'
    LIMIT 1
    """
    result = conn.execute(cols_query)

    # Get the column names from the description attribute
    column_names = [desc[0] for desc in result.description]

column_names

['year',
 'donor_code',
 'de_donorcode',
 'donor_name',
 'agency_code',
 'agency_name',
 'crs_id',
 'project_number',
 'initial_report',
 'recipient_code',
 'de_recipientcode',
 'recipient_name',
 'region_code',
 'de_regioncode',
 'region_name',
 'incomegroup_code',
 'de_incomegroup_code',
 'incomegroup_name',
 'flow_code',
 'flow_name',
 'bi_multi',
 'category',
 'finance_t',
 'aid_t',
 'usd_commitment',
 'usd_disbursement',
 'usd_received',
 'usd_commitment_defl',
 'usd_disbursement_defl',
 'usd_received_defl',
 'usd_adjustment',
 'usd_adjustment_defl',
 'usd_amount_untied',
 'usd_amount_partial_tied',
 'usd_amount_tied',
 'usd_amount_untied_defl',
 'usd_amount_partial_tied_defl',
 'usd_amounttied_defl',
 'usd_irtc',
 'usd_expert_commitment',
 'usd_expert_extended',
 'usd_export_credit',
 'currency_code',
 'commitment_national',
 'disbursement_national',
 'grant_equiv',
 'usd_grant_equiv',
 'short_description',
 'project_title',
 'purpose_code',
 'purpose_name',
 'sector_code',
 'sec

# DAC1 DATA

In [3]:
with duckdb.connect() as conn:

    cols_query = f"""
    SELECT * FROM '{dac1_file.as_posix()}'
    LIMIT 1
    """
    result = conn.execute(cols_query)

    # Get the column names from the description attribute
    column_names = [desc[0] for desc in result.description]

column_names

['DONOR',
 'Donor_1',
 'PART',
 'Part_1',
 'AIDTYPE',
 'Aid type',
 'FLOWS',
 'Fund flows',
 'AMOUNTTYPE',
 'Amount type',
 'TIME',
 'Year',
 'Value',
 'Flags']

## 2A All Donor ODA Trend

R code for 2A
TotalODAranking_previousyear <- DAC1data %>%
  filter(
    aid_type == "Official Development Assistance, grant equivalent measure",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year - 1
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year - 1,
    Donor = donor_name,
    `Total ODA` = Value / 1000,
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

TotalODAranking_endyear <- DAC1data %>%
  filter(
    aid_type == "Official Development Assistance, grant equivalent measure",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year,
    Donor = donor_name,
    `Total ODA` = Value / 1000,
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )
TotalODAranking <- rbind(TotalODAranking_endyear, TotalODAranking_previousyear)
TotalODAranking <- TotalODAranking[order(TotalODAranking$Year), ]

In [23]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2022 AND 2023
        AND "Amount type" = 'Current Prices (USD millions)'
        AND "Fund flows" = 'Grant equivalents'
        AND "Donor_1" IN {tuple(DAC_COUNTRIES)}
        AND "Aid type" = 'Official Development Assistance, grant equivalent measure'
    ), 

    ranked AS (  
        SELECT
            donor,
            year,
            value / 1000 AS "Total ODA", 
            row_number() OVER (PARTITION BY year ORDER BY value DESC) AS rn
        FROM base
    )

    SELECT
        year AS "Year",
        donor AS "Donor",
        round("Total ODA", 4) AS "Total ODA", 
        CASE 
            WHEN rn::TEXT LIKE '%1' AND rn != 11 THEN rn || 'st'
            WHEN rn::TEXT LIKE '%2' AND rn != 12 THEN rn || 'nd'
            WHEN rn::TEXT LIKE '%3' AND rn != 13THEN rn || 'rd'
            ELSE rn || 'th'
        END AS "Rank"
    FROM ranked
    ORDER BY year, rn
    """
    result = conn.execute(query).fetchdf()

result.to_csv(data_save_path / "DT_update" / "alldonor_2A_ODATrend.csv", index=False)

## 2B ODA GNI Ranking

ODAGNIranking_previousyear <- DAC1data %>%
  filter(
    aid_type == "ODA grant equivalent as percent of GNI",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year - 1
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year - 1,
    Donor = donor_name,
    `ODA as % GNI` = paste0(round(Value, 3), "%"),
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

ODAGNIranking_endyear <- DAC1data %>%
  filter(
    aid_type == "ODA grant equivalent as percent of GNI",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year,
    Donor = donor_name,
    `ODA as % GNI` = paste0(round(Value, 3), "%"),
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

ODAGNIranking <- rbind(ODAGNIranking_endyear, ODAGNIranking_previousyear)
ODAGNIranking <- ODAGNIranking[order(ODAGNIranking$Year), ]
print(ODAGNIranking)

In [24]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2022 AND 2023
        AND "Amount type" = 'Current Prices (USD millions)'
        AND "Fund flows" = 'Grant equivalents'
        AND "Donor_1" IN {tuple(DAC_COUNTRIES)}
        AND "Aid type" = 'ODA grant equivalent as percent of GNI'
    ), 

    ranked AS (  
        SELECT
            donor,
            year,
            round(value, 2) || '%' AS "ODA as % GNI", 
            row_number() OVER (PARTITION BY year ORDER BY value DESC) AS rn
        FROM base
    )

    SELECT
        year AS "Year",
        donor AS "Donor",
        "ODA as % GNI", 
        CASE 
            WHEN rn::TEXT LIKE '%1' AND rn != 11 THEN rn || 'st'
            WHEN rn::TEXT LIKE '%2' AND rn != 12 THEN rn || 'nd'
            WHEN rn::TEXT LIKE '%3' AND rn != 13THEN rn || 'rd'
            ELSE rn || 'th'
        END AS "Rank"
    FROM ranked
    ORDER BY year, rn
    """
    result = conn.execute(query).fetchdf()

#TODO: Slight rounding issues here. 
result.to_csv(data_save_path / "DT_update" / "alldonor_2B_ODAGNIranking.csv", index=False)

## 3A TOTAL ODA Over Time
CUT

## 3B Refugee

R code: 
donor_refugee_data <- DAC1data %>%
    filter(
      Year >= end_year - 4,
      donor_name == current_donor,
      amount_type == "Constant Prices",
      flow_type == "Net Disbursements",
      aid_type %in% c(
        "I.A.8.2. Refugees in donor countries",
        "I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)",
        "I.B.1.2. EU institutions",
        "I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)"
      )
    ) %>%
    select(aid_type, Year, Value) %>%
    group_by(Year) %>%
    summarise(
      `ODA for Development Priorities` = sum(Value * case_when(
        aid_type %in% c("I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)") ~ 1,
        aid_type %in% c("I.A.8.2. Refugees in donor countries") ~ -1,
        aid_type %in% c("I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)") ~ 1,
        aid_type %in% c("I.B.1.2. EU institutions") ~ -1,
        TRUE ~ 0
      )),
      `Contributions to EUI` = sum(Value * case_when(
        aid_type %in% c("I.B.1.2. EU institutions") ~ 1,
        TRUE ~ 0
      )),
      `In-donor Refugee Costs` = sum(Value * case_when(
        aid_type %in% c("I.A.8.2. Refugees in donor countries") ~ 1,
        TRUE ~ 0
      ))
    )


In [29]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2019 AND 2023
        AND "Amount type" = 'Constant Prices (2022 USD millions)'
        AND "Fund flows" = 'Net Disbursements'
        AND "Aid type" IN (
            'I.A.8.2. Refugees in donor countries',
            'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)',
            'I.B.1.2. EU institutions',
            'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)'
        )
    ), 

    filtered AS (
    SELECT
        donor,
        year,  
        SUM(value * 
        CASE 
            WHEN aid_type = 'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)' THEN 1
            WHEN aid_type = 'I.A.8.2. Refugees in donor countries' THEN -1
            WHEN aid_type = 'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)' THEN 1
            WHEN aid_type = 'I.B.1.2. EU institutions' THEN -1
            ELSE 0
        END
        ) AS "ODA for Development Priorities", 
        SUM(value *
        CASE 
            WHEN aid_type = 'I.B.1.2. EU institutions' THEN 1
            ELSE 0
        END
        ) AS "Contributions to EUI", 
        SUM(value *
        CASE
            WHEN aid_type = 'I.A.8.2. Refugees in donor countries' THEN 1
            ELSE 0
        END
        ) AS "In-donor Refugee Costs"
    FROM base
    GROUP BY donor, year

    )
    SELECT
        donor,
        year AS "Year",
        "ODA for Development Priorities",
        "Contributions to EUI",
        "In-donor Refugee Costs"
    FROM filtered f
    ORDER BY year, donor
    """
    result = conn.execute(query).fetchdf()


for donor in DAC_COUNTRIES:
    if donor in ['Austria', 'Belgium', 'Denmark', 'Finland', 'Ireland', 'Luxembourg', 'Switzerland']:
        save_path = data_save_path / "OP" / f"{donor}_3B_refugee.csv"
    else: 
        save_path = data_save_path / "DT_update" / f"{donor}_3B_refugee.csv"
    
    data = result[result["donor"] == donor][['Year', 'ODA for Development Priorities', 'Contributions to EUI', 'In-donor Refugee Costs']]
    data.to_csv(save_path, index=False)
    
# result[result["donor"] == "Austria"][['Year', 'ODA for Development Priorities', 'Contributions to EUI', 'In-donor Refugee Costs']].to_csv("Austria_3B_refugee.csv", index=False)

['Australia', 'Austria', 'Azerbaijan', 'Belgium', 'Bulgaria', 'Canada', 'Chinese Taipei', 'Croatia', 'Cyprus', 'Czechia', 'DAC Countries, Total', 'DAC EU Members + EC, Total', 'DAC EU Members, Total', 'Denmark', 'EU Institutions', 'Estonia', 'Finland', 'France', 'G7 Countries, Total', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Korea', 'Kuwait', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Multilaterals, Total', 'Netherlands', 'New Zealand', 'Non-DAC Countries, Total', 'Norway', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Saudi Arabia', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Türkiye', 'United Arab Emirates', 'United Kingdom', 'United States', 'Monaco']


## 4 BI-MULTI

R code for Bi-multi
for (current_donor in specified_countries) {
  donor_multi_bi_data <- DAC1data %>% 
    filter(Year >= end_year_dac - 4) %>%
    filter(
      Year <= end_year_dac,
      donor_name == current_donor,
      amount_type == "Constant Prices",
      flow_type == "Gross Disbursements",
      aid_type %in% c(
        "I. Official Development Assistance (ODA) (I.A + I.B)",
        "I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)",
        "I.A. Memo: ODA channelled through multilateral organisations",
        "I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)")) %>%
    select(aid_type, Year, Value) %>%
    summarise(
      `Year` = Year[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Bilateral funding` = Value[aid_type %in% c("I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)")] - Value[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Bilateral as earmarked funding through multilaterals` = Value[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Multilateral as core contributions to organizations` = Value[aid_type %in% c("I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)")],
      `Total ODA` = Value[aid_type %in% c("I. Official Development Assistance (ODA) (I.A + I.B)")]
    ) %>%
    mutate(
      `Bilateral` = paste0(round(`Bilateral funding` / `Total ODA` * 100), "%"),
      `Earmarked` = paste0(round(`Bilateral as earmarked funding through multilaterals` / `Total ODA` * 100), "%"),
      `Multilateral` = paste0(round(`Multilateral as core contributions to organizations` / `Total ODA` * 100), "%")
    ) %>%
    select(`Year`, `Bilateral funding`, `Bilateral as earmarked funding through multilaterals`, `Multilateral as core contributions to organizations`, `Bilateral`, `Earmarked`, `Multilateral`)
  #print(donor_multi_bi_data)

In [ ]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2018 AND 2023
        AND "Amount type" = 'Constant Prices (2022 USD millions)'
        AND "Fund flows" = 'Gross Disbursements'
        AND "Aid type" IN (
            'I. Official Development Assistance (ODA) (I.A + I.B)',
            'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)',
            'I.A. Memo: ODA channelled through multilateral organisations',
            'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)'
        )
    ), 

    filtered AS (  
        SELECT
            donor,
            year,
            SUM(CASE WHEN aid_type = 'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)' THEN "value" END) -
            SUM(CASE WHEN aid_type = 'I.A. Memo: ODA channelled through multilateral organisations' THEN "value" END) AS "Bilateral funding",
            SUM(CASE WHEN aid_type = 'I.A. Memo: ODA channelled through multilateral organisations' THEN "value" END) AS "Bilateral as earmarked funding through multilaterals",
            SUM(CASE WHEN aid_type = 'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)' THEN "value" END) AS "Multilateral as core contributions to organizations",
            SUM(CASE WHEN aid_type = 'I. Official Development Assistance (ODA) (I.A + I.B)' THEN "value" END) AS "Total ODA"
        FROM base
        GROUP BY 1,2
    )
    SELECT
        donor,
        year AS "Year",
        "Bilateral funding",
        "Bilateral as earmarked funding through multilaterals",
        "Multilateral as core contributions to organizations",
        CONCAT(ROUND("Bilateral funding" / "Total ODA" * 100), '%') AS "Bilateral",
        CONCAT(ROUND("Bilateral as earmarked funding through multilaterals" / "Total ODA" * 100), '%') AS "Earmarked",
        CONCAT(ROUND("Multilateral as core contributions to organizations" / "Total ODA" * 100), '%') AS "Multilateral"
    FROM filtered f
    """
    result = conn.execute(query).fetchdf()

result[result["donor"] == "Australia"][]

#TODO: Make a function that generates a CSV for every DAC country

,donor,Year,Bilateral funding,Bilateral as earmarked funding through multilaterals,Multilateral as core contributions to organizations,Bilateral,Earmarked,Multilateral
40,Australia,2021,2085.961820,984.738237,477.054689,59.0%,28.0%,13.0%
85,Australia,2019,1977.750286,582.003295,760.182299,60.0%,18.0%,23.0%
99,Australia,2022,1896.860000,726.270000,461.470000,61.0%,24.0%,15.0%
183,Australia,2018,2157.825425,651.383917,658.545609,62.0%,19.0%,19.0%
198,Australia,2023,NaN,NaN,NaN,%,%,%
323,Australia,2020,1876.707397,785.288504,631.144253,57.0%,24.0%,19.0%


In [72]:
with duckdb.connect() as conn:
    # Run a query on the Parquet file
    query = f"""
    SELECT DISTINCT
        "Year"
    FROM '{dac1_file.as_posix()}'
    """
    result = conn.execute(query).fetchdf()

print(result)

    Year
0   1976
1   1962
2   2001
3   2009
4   2016
..   ...
59  1985
60  1998
61  1970
62  1964
63  1978

[64 rows x 1 columns]
